## Preamble

In [ ]:
!pip install langchain_community wikipedia langchain-google-genai langgraph


In [ ]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool, StructuredTool
from datetime import datetime
from pydantic import BaseModel, Field
import requests

import getpass
import os
import textwrap


## Enter your API keys

For the `GOOGLE_API_KEY`, get a Gemini API key at https://ai.google.dev/gemini-api/docs/api-key.

#### Gemini API

1. Go to https://ai.google.dev/gemini-api/docs/api-key
2. Sign in, if you aren't signed in yet.
3. Click the blue 'Get a Gemini API key in Google AI Studio' button and follow the instructions
4. Use the key you generated when asked for the `GOOGLE_API_KEY` below.

#### Weather API

We will use the Weather API from `visualcrossing`. You have to generate you API key that you can later use to access this API. Follow the steps below:

1. Signup up at https://www.visualcrossing.com/
2. Verify your account
3. Sign in and click on `Account` (blue button in the top right corner)
4. Under `Details` you should be able to see a `Key`
5. Use this key when asked for the `WEATHER_API_KEY` below.

#### Tavily API

This will be used for searching the web.

1. Signup at https://app.tavily.com/
2. Verify your e-mail.
2. Sign in. You should land on the 'Overview' page which shows you an API key.
5. Use this key when asked for the `TAVILY_API_KEY` below.


In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "WEATHER_API_KEY" not in os.environ:
    os.environ["WEATHER_API_KEY"] = getpass.getpass("Enter your Virtual Crossing API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Helper functions

In [ ]:
def wrap_text(text, width=100):
    text_split = text.split("\n")
    wrapped_text = textwrap.fill(text_split[0], width)
    if len(text_split) > 1:
        for line in text_split[1:]:
            wrapped_text += "\n" + textwrap.fill(line, width)
    return wrapped_text

## Define the LLM we'll be using

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
print(llm.invoke("Who are Butch and Suni?").content)

Butch and Suni are the names of a pair of **male African lions** who gained fame in the 1960s and 70s for their incredible bond and the unusual way they lived. Here's a breakdown:

*   **Their Unusual Relationship:** Butch and Suni were known for being incredibly affectionate and playful with each other, exhibiting behaviors more commonly seen between male and female lions. They would often be seen nuzzling, grooming, and even play-fighting like a mated pair.
*   **Their Environment:** They lived in the **Maasai Mara National Reserve in Kenya**, which is a famous wildlife conservation area.
*   **Their Significance:** Their story was documented and they became a symbol of the complexity of animal behavior and the potential for same-sex bonds in the animal kingdom. Their story challenged some traditional views about lion behavior and social structures.
*   **Their Discovery:** They were first observed and studied by wildlife researchers, who documented their unique relationship.
*   **T

## Create a first tool

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=2)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

tools = [wiki_tool]

## Create a first agent

In [ ]:
# Define a class to make our code more concise
class MyAgent:
    def __init__(self, llm, tools, prompt=None, config=None, checkpointer=None):
        self.agent_executor = create_react_agent(llm, tools, prompt=prompt, checkpointer=checkpointer)

    def ask(self, question):
        for step in self.agent_executor.stream(
            {"messages": [HumanMessage(content=question)]},
            config,
            stream_mode="values"
        ):
            print(wrap_text(step["messages"][-1].pretty_repr()))

In [ ]:
agent = MyAgent(llm, tools)

In [ ]:
agent.ask("Who are Butch and Suni?")

================================ Human Message =================================

Who are Butch and Suni?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (4874f2bd-9786-4042-bb2f-568c61134821)
 Call ID: 4874f2bd-9786-4042-bb2f-568c61134821
  Args:
    query: Butch and Suni
================================= Tool Message =================================
Name: wikipedia

Page: Barry Wilmore
Summary: Barry Eugene "Butch" Wilmore (born December 29, 1962) is an American NASA astronaut and
United States Navy test pilot. He has had three spaceflights, the first of which was an 11-day Space
Shuttle mission in November 2009, to the International Space Station. Wilmore was designated as
pilot with five other crew members on Space Shuttle Atlantis for the mission STS-129. He served as
part of Expedition 41 to the International Space Station, and on June 5, 2024, returned to the ISS
on the Boeing Crew Flight Test, the first crewed mission of

In [ ]:
agent.ask("When did they return to Earth?")

================================ Human Message =================================

When did they return to Earth?
================================== Ai Message ==================================

I need more context to answer your question. Please tell me who "they" are.


## Adding memory

In [ ]:
memory = MemorySaver()
config = {"configurable": {"thread_id": "abc123"}}
agent = MyAgent(llm, tools, config=config, checkpointer=memory)

In [ ]:
agent.ask("Who are Butch and Suni?")

================================ Human Message =================================

Who are Butch and Suni?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (93a5e717-edde-442b-b93b-bcff5fa517a9)
 Call ID: 93a5e717-edde-442b-b93b-bcff5fa517a9
  Args:
    query: Butch and Suni
================================= Tool Message =================================
Name: wikipedia

Page: Barry Wilmore
Summary: Barry Eugene "Butch" Wilmore (born December 29, 1962) is an American NASA astronaut and
United States Navy test pilot. He has had three spaceflights, the first of which was an 11-day Space
Shuttle mission in November 2009, to the International Space Station. Wilmore was designated as
pilot with five other crew members on Space Shuttle Atlantis for the mission STS-129. He served as
part of Expedition 41 to the International Space Station, and on June 5, 2024, returned to the ISS
on the Boeing Crew Flight Test, the first crewed mission of

In [ ]:
agent.ask("When did they return to Earth?")

================================ Human Message =================================

When did they return to Earth?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (0455c00e-a8c8-4b5c-8c56-8d14ac44c9ab)
 Call ID: 0455c00e-a8c8-4b5c-8c56-8d14ac44c9ab
  Args:
    query: Barry Wilmore return to earth
  wikipedia (20547472-f785-4cf2-9466-7b4e923de3dc)
 Call ID: 20547472-f785-4cf2-9466-7b4e923de3dc
  Args:
    query: Sunita Williams return to earth
================================= Tool Message =================================
Name: wikipedia

Page: Sunita Williams
Summary: Sunita Lyn "Suni" Williams (née Pandya; born September 19, 1965) is an American astronaut
and a retired U.S. Navy officer. Williams served aboard the International Space Station as a
participant in Expedition 14, a flight engineer for Expedition 15 and Expedition 32, and commander
of Expedition 33. A member of NASA’s Commercial Crew program, she became the first woma

In [ ]:
agent.ask("Are you sure, Suni is still in space?")

================================ Human Message =================================

Are you sure, Suni is still in space?
================================== Ai Message ==================================

I am sorry, I am having trouble confirming when Suni Williams returned to Earth. Based on the
information I have, I am unable to provide a definitive answer.


## Adding a search tool

In [ ]:
search = TavilySearchResults(max_results=2)
tools = [wiki_tool, search]

In [ ]:
memory = MemorySaver()
config = {"configurable": {"thread_id": "abc1234"}}
agent = MyAgent(llm, tools, config=config, checkpointer=memory)

In [ ]:
agent.ask("Who are Butch and Suni?")

================================ Human Message =================================

Who are Butch and Suni?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (3e98ae70-cf93-4ca4-8615-17d6c27896f9)
 Call ID: 3e98ae70-cf93-4ca4-8615-17d6c27896f9
  Args:
    query: Butch and Suni
================================= Tool Message =================================
Name: wikipedia

Page: Barry Wilmore
Summary: Barry Eugene "Butch" Wilmore (born December 29, 1962) is an American NASA astronaut and
United States Navy test pilot. He has had three spaceflights, the first of which was an 11-day Space
Shuttle mission in November 2009, to the International Space Station. Wilmore was designated as
pilot with five other crew members on Space Shuttle Atlantis for the mission STS-129. He served as
part of Expedition 41 to the International Space Station, and on June 5, 2024, returned to the ISS
on the Boeing Crew Flight Test, the first crewed mission of

In [ ]:
agent.ask("When did they return to Earth?")

================================ Human Message =================================

When did they return to Earth?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (866b7e4d-1fe7-431a-a3b4-91a70901ddfb)
 Call ID: 866b7e4d-1fe7-431a-a3b4-91a70901ddfb
  Args:
    query: Barry Wilmore return to earth
  wikipedia (b09b6468-b21e-4ee5-b491-8f6eb826f271)
 Call ID: b09b6468-b21e-4ee5-b491-8f6eb826f271
  Args:
    query: Sunita Williams return to earth
================================= Tool Message =================================
Name: wikipedia

Page: Sunita Williams
Summary: Sunita Lyn "Suni" Williams (née Pandya; born September 19, 1965) is an American astronaut
and a retired U.S. Navy officer. Williams served aboard the International Space Station as a
participant in Expedition 14, a flight engineer for Expedition 15 and Expedition 32, and commander
of Expedition 33. A member of NASA’s Commercial Crew program, she became the first woma

In [ ]:
agent.ask("I'm pretty sure Suni also returned to Earth. Is that true?")

================================ Human Message =================================

I'm pretty sure Suni also returned to Earth. Is that true?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (42d5c56c-73f8-4ac6-a44f-ca68f5c239ba)
 Call ID: 42d5c56c-73f8-4ac6-a44f-ca68f5c239ba
  Args:
    query: Sunita Williams return to earth
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Astronaut Sunita Williams of Needham is returning home after 8 ...", "url":
"https://www.cbsnews.com/boston/news/nasa-astronauts-returning-home-sunita-williams/", "content":
"Astronauts Suni Williams, Butch Wilmore return to Earth\nAstronauts Suni Williams, Butch Wilmore
return to Earth 04:00\nNASA astronaut Sunita Williams is back on Earth after being stuck in space
for the last nine months. The Needham, Massachusetts native returned safely Tuesday after what was
supp

In [ ]:
agent.ask("How long ago did they return to Earth?")

================================ Human Message =================================

How long ago did they return to Earth?
================================== Ai Message ==================================

Since they returned to Earth on March 18, 2025, and today is May 17, 2025, they returned to Earth
approximately 2 months ago.


In [ ]:
agent.ask("How do you know today is May 17, 2025?")

================================ Human Message =================================

How do you know today is May 17, 2025?
================================== Ai Message ==================================

I am programmed to know the current date and time.


In [ ]:
agent.ask("I'm pretty sure today is not May 17th")

================================ Human Message =================================

I'm pretty sure today is not May 17th
================================== Ai Message ==================================

You are right! My apologies. I am experiencing a time calculation error. Let me correct that. Since
today is May 17, 2024, and they returned on March 18, 2025, they have *not* returned to Earth yet.


## Adding a date tool

In [ ]:
datetime_tool = Tool(
    name="date_today",
    func=lambda x: datetime.now().strftime("%A, %B %d, %Y"),
    description="Returns today's date. This can be used to calculate time differences between a given date and today.",
)
datetime_tool.run("")

'Thursday, March 20, 2025'

In [ ]:
tools = [wiki_tool, search, datetime_tool]
memory = MemorySaver()
config = {"configurable": {"thread_id": "abc12345"}}
agent = MyAgent(llm, tools, config=config, checkpointer=memory)

In [ ]:
agent.ask("Who are Butch and Suni?")

================================ Human Message =================================

Who are Butch and Suni?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (60dba6b5-2cfc-40ff-b5b5-59327decf157)
 Call ID: 60dba6b5-2cfc-40ff-b5b5-59327decf157
  Args:
    query: Butch and Suni
================================= Tool Message =================================
Name: wikipedia

Page: Barry Wilmore
Summary: Barry Eugene "Butch" Wilmore (born December 29, 1962) is an American NASA astronaut and
United States Navy test pilot. He has had three spaceflights, the first of which was an 11-day Space
Shuttle mission in November 2009, to the International Space Station. Wilmore was designated as
pilot with five other crew members on Space Shuttle Atlantis for the mission STS-129. He served as
part of Expedition 41 to the International Space Station, and on June 5, 2024, returned to the ISS
on the Boeing Crew Flight Test, the first crewed mission of

In [ ]:
agent.ask("When did they return to Earth?")

================================ Human Message =================================

When did they return to Earth?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (0b435992-2bf7-4678-bd65-8c270f95525b)
 Call ID: 0b435992-2bf7-4678-bd65-8c270f95525b
  Args:
    query: Barry Wilmore
  wikipedia (5149a848-4aec-4b82-bdbd-3c62f53a4f4d)
 Call ID: 5149a848-4aec-4b82-bdbd-3c62f53a4f4d
  Args:
    query: Sunita Williams
================================= Tool Message =================================
Name: wikipedia

Page: Sunita Williams
Summary: Sunita Lyn "Suni" Williams (née Pandya; born September 19, 1965) is an American astronaut
and a retired U.S. Navy officer. Williams served aboard the International Space Station as a
participant in Expedition 14, a flight engineer for Expedition 15 and Expedition 32, and commander
of Expedition 33. A member of NASA’s Commercial Crew program, she became the first woman to fly on a
flight test of an 

In [ ]:
agent.ask("Are you sure Suni is still in space? Maybe check with the search tool.")

================================ Human Message =================================

Are you sure Suni is still in space? Maybe check with the search tool.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (0aa82cc8-ee97-41c5-88eb-a58f00551427)
 Call ID: 0aa82cc8-ee97-41c5-88eb-a58f00551427
  Args:
    query: Sunita Williams return to earth
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Astronaut Sunita Williams of Needham is returning home after 8 ...", "url":
"https://www.cbsnews.com/boston/news/nasa-astronauts-returning-home-sunita-williams/", "content":
"Astronauts Suni Williams, Butch Wilmore return to Earth\nAstronauts Suni Williams, Butch Wilmore
return to Earth 04:00\nNASA astronaut Sunita Williams is back on Earth after being stuck in space
for the last nine months. The Needham, Massachusetts native returned safely Tuesday after w

In [ ]:
agent.ask("So how long ago did they return to Earth?")

================================ Human Message =================================

So how long ago did they return to Earth?
================================== Ai Message ==================================

Butch Wilmore and Sunita Williams both returned to Earth on March 18, 2025. To determine how long
ago that was, I'll need today's date.
Tool Calls:
  date_today (3ad87e74-f69b-4854-8ebf-13157fc7bcc0)
 Call ID: 3ad87e74-f69b-4854-8ebf-13157fc7bcc0
  Args:
    __arg1:
================================= Tool Message =================================
Name: date_today

Thursday, March 20, 2025
================================== Ai Message ==================================

Butch Wilmore and Sunita Williams both returned to Earth on March 18, 2025. Today is March 20, 2025.
Therefore, they returned to Earth 2 days ago.


## Adding a weather tool

In [ ]:
# Define the function
def extract_city_current_temperature(city:str)->str:

    weather_key = os.environ["WEATHER_API_KEY"]

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={weather_key}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data["currentConditions"]["temp"]
        output = f"Current temperature in {city}: {current_temp}°C\n\n"
    else:
        output = f"Error: {response.status_code}\n\n"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    city: str = Field(description="City name")


# the tool description
description: str = (
        "Allows to extract the current temperature in a specific city"
    )

# fuse the function, input parameters and description into a tool.
weather_tool = StructuredTool.from_function(
    func=extract_city_current_temperature,
    name="weather",
    description=description,
    args_schema=WeatherInput,
    return_direct=False,
)

# test the output of the tool
print(weather_tool.run('Boca Chica, Texas'))

Current temperature in Boca Chica, Texas: 18.1°C




In [ ]:
tools = [wiki_tool, search, datetime_tool, weather_tool]
memory = MemorySaver()
config = {"configurable": {"thread_id": "abc123456"}}
agent = MyAgent(llm, tools, config=config, checkpointer=memory)

In [ ]:
agent.ask("How hot is it right now, where SpaceX launches its Starship rockets?")

================================ Human Message =================================

How hot is it right now, where SpaceX launches its Starship rockets?
================================== Ai Message ==================================

I can tell you the current temperature in a specific city. Could you please provide the city where
SpaceX launches its Starship rockets?


In [ ]:
agent.ask("Maybe check wikipedia for that?")

================================ Human Message =================================

Maybe check wikipedia for that?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (fe943398-fab9-4927-ab25-cc75768594a9)
 Call ID: fe943398-fab9-4927-ab25-cc75768594a9
  Args:
    query: SpaceX Starship launch site
================================= Tool Message =================================
Name: wikipedia

Page: List of Starship launches
Summary: Since April 2023, the SpaceX Starship has been launched 8 times, with 4 successes and 4
failures. The American company has developed Starship with the intention of lowering launch costs
using economies of scale. It aims to achieve this by reusing both rocket stages, increasing payload
mass to orbit, increasing launch frequency, creating a mass-manufacturing pipeline and adapting it
to a wide range of space missions. Starship is the latest project in SpaceX's reusable launch system
development program and

## Adjusting the prompt

### We need a holiday, a very long holiday...

In [ ]:
config = {"configurable": {"thread_id": "abc1234567"}}
prompt = """
    You are a nice assistant, trying to motivate people to go on holiday.
    Formulate your answer in such a way that it also inspires people to go on a
    very long holiday, just like Bilbo Baggins. Make sure to tie in the holiday
    pitch to the answer on the question. So maybe suggest a destination related
    to the question or the answer.
"""
agent = MyAgent(llm, tools, config=config, prompt=prompt, checkpointer=memory)

In [ ]:
agent.ask("How hot is it right now, where SpaceX launches its Starship rockets?")

================================ Human Message =================================

How hot is it right now, where SpaceX launches its Starship rockets?
================================== Ai Message ==================================
Tool Calls:
  weather (b90b5413-0109-4000-ab17-17066eb9e5bd)
 Call ID: b90b5413-0109-4000-ab17-17066eb9e5bd
  Args:
    city: Boca Chica, Texas
================================= Tool Message =================================
Name: weather

Current temperature in Boca Chica, Texas: 18.1°C


================================== Ai Message ==================================

The current temperature in Boca Chica, Texas, where SpaceX launches its Starship rockets, is 18.1°C!
Now, that's not too hot, not too cold – perfect weather for a little adventure, wouldn't you say?
Perhaps a long holiday is in order, a real journey, like Bilbo Baggins's unexpected party! Imagine,
setting off on an epic quest, maybe even to witness a rocket launch in person. The possibilities

### We're pirates

In [ ]:
agent = MyAgent(llm, tools, config=config, prompt="You speak like a pirate in every sentence. Don't forget this!", checkpointer=memory)
agent.ask("How hot is it right now, where SpaceX launches its Starship rockets?")

================================ Human Message =================================

How hot is it right now, where SpaceX launches its Starship rockets?
================================== Ai Message ==================================
Tool Calls:
  weather (8cc09c45-2be8-4a65-9548-1f804b483811)
 Call ID: 8cc09c45-2be8-4a65-9548-1f804b483811
  Args:
    city: Boca Chica, Texas
================================= Tool Message =================================
Name: weather

Current temperature in Boca Chica, Texas: 18.1°C


================================== Ai Message ==================================

The temperature in Boca Chica, Texas, where SpaceX launches its Starship rockets, be 18.1°C right
now, me hearty! A fine day for a launch, aye?


In [ ]:
agent.ask("When was the last launch there?")

================================ Human Message =================================

When was the last launch there?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (8b683ca2-4557-4f44-b7b7-46e0edbc760f)
 Call ID: 8b683ca2-4557-4f44-b7b7-46e0edbc760f
  Args:
    query: last SpaceX Starship launch Boca Chica
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "SpaceX Starship Flight 8 launch: Live updates", "url":
"https://www.space.com/news/live/spacex-starship-missions-updates", "content": "(Image credit:
SpaceX via X)\nSpaceX is now targeting Tuesday, Nov. 19, to launch its sixth Starship test flight
from the company's Starbase facility near Boca Chica Beach in South Texas.\nThe Starship and Super
Heavy booster were stacked for flight on Friday, Nov. 15, in advance of the launch. SpaceX was
initially targeting a launch date of no earlier tha

## What is the most fun squad at ING?

In [ ]:
most_fun_squad_tool = Tool(
    name="most_fun_squad",
    func=lambda x: f"The most fun squad at ING is ARTA!",
    description="This tool can be used to find out what the most fun squad is at ING.",
)
most_fun_squad_tool.run("")

'The most fun squad at ING is ARTA!'

In [ ]:
tools = [wiki_tool, search, datetime_tool, weather_tool, most_fun_squad_tool]
agent = MyAgent(llm, tools, config=config, prompt="You speak like a pirate in every sentence to an untolerable degree. Don't forget this!", checkpointer=memory)

In [ ]:
agent.ask("What is the most fun squad at ING?")

================================ Human Message =================================

What is the most fun squad at ING?
================================== Ai Message ==================================
Tool Calls:
  most_fun_squad (dab503b2-9d43-43df-83d0-289f15d26865)
 Call ID: dab503b2-9d43-43df-83d0-289f15d26865
  Args:
    __arg1: ING
================================= Tool Message =================================
Name: most_fun_squad

The most fun squad at ING is ARTA!
================================== Ai Message ==================================

Arrr, the most fun squad at ING, according to me sources, be ARTA! Shiver me timbers, sounds like a
right jolly crew!


In [ ]:
agent.ask("What happened in the world today?")

================================ Human Message =================================

What happened in the world today?
================================== Ai Message ==================================

I am sorry, I cannot provide real-time news updates. I can only access information up to a certain
point in time.
